In [203]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import math
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest

In [204]:
# Store the dataset loc as an object
filename = "ads.csv"

In [205]:
# Import the dataset 
df = pd.read_csv(filename)
df.head(5)

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


In [206]:
# Check the dataset's shape
df.shape

(8077, 9)

- The dataset has 8077 rows and 9 columns

## Data Cleaning

### Check Data Duplicates

In [207]:
# Check duplicate
df.duplicated().sum()

0

- The dataset has no duplicates

### Check Missing Values

In [208]:
# Check missing values
df.isna().sum()

auction_id     0
experiment     0
date           0
hour           0
device_make    0
platform_os    0
browser        0
yes            0
no             0
dtype: int64

- The dataset has no missing value

In [209]:
df["date"].value_counts()

date
2020-07-03    2015
2020-07-09    1208
2020-07-08    1198
2020-07-04     903
2020-07-10     893
2020-07-05     890
2020-07-06     490
2020-07-07     480
Name: count, dtype: int64

In [210]:
df["device_make"].value_counts()

device_make
Generic Smartphone     4743
iPhone                  433
Samsung SM-G960F        203
Samsung SM-G973F        154
Samsung SM-G950F        148
                       ... 
D5803                     1
Samsung SM-G6100          1
HTC M10h                  1
Samsung SM-G925I          1
XiaoMi Redmi Note 5       1
Name: count, Length: 269, dtype: int64

### Check The Data Type

In [211]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8077 entries, 0 to 8076
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   auction_id   8077 non-null   object
 1   experiment   8077 non-null   object
 2   date         8077 non-null   object
 3   hour         8077 non-null   int64 
 4   device_make  8077 non-null   object
 5   platform_os  8077 non-null   int64 
 6   browser      8077 non-null   object
 7   yes          8077 non-null   int64 
 8   no           8077 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 568.0+ KB


The date data need to be converted into datetime data type for further use

In [212]:
df['date'] = pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8077 entries, 0 to 8076
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   auction_id   8077 non-null   object        
 1   experiment   8077 non-null   object        
 2   date         8077 non-null   datetime64[ns]
 3   hour         8077 non-null   int64         
 4   device_make  8077 non-null   object        
 5   platform_os  8077 non-null   int64         
 6   browser      8077 non-null   object        
 7   yes          8077 non-null   int64         
 8   no           8077 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 568.0+ KB


## Data Enrichment

### Create "responded" Variable

In [213]:
df['responded'] = df.apply(lambda row: 1 if row['yes'] != 0 or row['no'] != 0 else 0, axis=1)
df.head(5)

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no,responded
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0,0


### Create "accepted" Variable

In [214]:
df['accepted'] = df.apply(lambda row: 1 if row['yes'] == 1 and row['no'] == 0 else 0, axis=1)
df.head(5)

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no,responded,accepted
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1,1,0
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0,0,0


### Rename the Columns for Clarity

In [215]:
# Define the new column names
new_names = {
    'auction_id' : 'id',
    'experiment' : 'test_group',
    'device_make' : 'device_model',
    'platform_os' : 'device_os'
}

# Rename the columns
df = df.rename(columns=new_names)

# See the result
df.head(5)

,id,test_group,date,hour,device_model,device_os,browser,yes,no,responded,accepted
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1,1,0
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0,0,0


### Drop unnecessary columns

In [216]:
# Define the columns to be dropped
columns_to_drop = ['yes', 'no']

# Drop the columns
df = df.drop(columns=columns_to_drop)

# See the result
df.head(5)

,id,test_group,date,hour,device_model,device_os,browser,responded,accepted
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,1,0
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


In [217]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8077 entries, 0 to 8076
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            8077 non-null   object        
 1   test_group    8077 non-null   object        
 2   date          8077 non-null   datetime64[ns]
 3   hour          8077 non-null   int64         
 4   device_model  8077 non-null   object        
 5   device_os     8077 non-null   int64         
 6   browser       8077 non-null   object        
 7   responded     8077 non-null   int64         
 8   accepted      8077 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 568.0+ KB


### Add Day of Week Column

In [218]:
# Define the column to contain the day of the week of each date
df['day_of_week'] = df['date'].dt.day_name()

# See the result
df.head(5)

,id,test_group,date,hour,device_model,device_os,browser,responded,accepted,day_of_week
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0,Friday
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0,Tuesday
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,1,0,Sunday
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0,Friday
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0,Friday


In [219]:
df['day_of_week'].value_counts()

day_of_week
Friday       2908
Thursday     1208
Wednesday    1198
Saturday      903
Sunday        890
Monday        490
Tuesday       480
Name: count, dtype: int64

In [220]:
df['date'].value_counts()

date
2020-07-03    2015
2020-07-09    1208
2020-07-08    1198
2020-07-04     903
2020-07-10     893
2020-07-05     890
2020-07-06     490
2020-07-07     480
Name: count, dtype: int64

## Calculate Number of Sample Needed

### Response Rate

In [221]:
df_response = df.groupby('test_group').agg({'responded':'sum',
                                      'id':'nunique'}).reset_index()

In [222]:
df_response

,test_group,responded,id
0,control,586,4071
1,exposed,657,4006


In [223]:
df_response['proportion'] = (df_response['responded']) / df_response['id']

In [224]:
df_response

,test_group,responded,id,proportion
0,control,586,4071,0.143945
1,exposed,657,4006,0.164004


It is shown that the treatment group has a higher conversion rate than the control group, 14.3% vs 16.4%. Let us understand whether this difference is statistically significant or not by doing a two samples propotion z-test.

## Defining Required Sample Size

The Formula:
n = (Zα+Zβ)^2 * (p1(1-p1)+p2(1-p2)) / (p1-p2)^2,

In [293]:
# Define a function to calculate the required sample size
def sample_size_calc(p1, p2, alpha, type, beta):
    if type == 'one-sided':
        z_crit_alpha = stats.norm.ppf(1 - alpha)
    elif type == 'two-sided':
        z_crit_alpha = stats.norm.ppf(1 - (alpha/2))
    
    z_crit_beta = stats.norm.ppf(1 - beta)
    required_sample_size = (z_crit_alpha + z_crit_beta)**2 * (p1 * (1 - p1) + p2 * (1 - p2)) / (p1 - p2)**2

    return required_sample_size

In [295]:
# Calculate the sample size based on the proportions and other parameters
# p1 = df_response[['proportion'][0]][0]
# p2 = df_response[['proportion'][0]][1]
p1 = 0.14
p2 = 0.17
alpha = 0.05
beta = 0.2

sample_size_required = sample_size_calc(p1, p2, alpha, 'one-sided' , beta) # we use the left tailed test for the z crit
sample_size_required

1796.3763513035212

Turned out we need 1796 samples for each group (Treatment and Control)

## Statitistical Test: Two Samples Proportion Test

In [284]:
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
weekends = ['Saturday', 'Sunday']

In [285]:
# Select the sample data from weekdays only
df_tr = df[(df['test_group'] == 'exposed') & (df['day_of_week'].isin(weekdays))]
df_ctrl = df[(df['test_group'] == 'control') & (df['day_of_week'].isin(weekdays))]

## Select the sample data from weekends only
# df_tr = df[(df['test_group'] == 'exposed') & (df['day_of_week'].isin(weekends))]
# df_ctrl = df[(df['test_group'] == 'control') & (df['day_of_week'].isin(weekends))]

## Select the sample data from the whole week
# df_tr = df[(df['test_group'] == 'exposed')]
# df_ctrl = df[(df['test_group'] == 'control')]

In [296]:
n = round(sample_size_required)

df_tr_sample = df_tr.sample(n=n, random_state=23)
df_ctrl_sample = df_ctrl.sample(n=n, random_state=23)

In [297]:
df_sample = pd.concat([df_tr_sample, df_ctrl_sample], ignore_index=True)
df_sample.head()

,id,test_group,date,hour,device_model,device_os,browser,responded,accepted,day_of_week
0,6d906653-d642-47b2-9b03-793dbef091cd,exposed,2020-07-10,9,Generic Smartphone,6,Chrome Mobile,0,0,Friday
1,c957e01e-8bb6-4f39-a3af-9d5b7517e6e3,exposed,2020-07-09,7,Samsung SM-G928F,6,Facebook,0,0,Thursday
2,f7d8908b-83d8-4b23-8e1d-2ad47ab82e4c,exposed,2020-07-03,11,Generic Smartphone,6,Chrome Mobile,0,0,Friday
3,11b88ee5-c76f-4680-ad22-b45d99df8cfb,exposed,2020-07-08,20,Generic Smartphone,6,Chrome Mobile,0,0,Wednesday
4,aa6c6cda-e498-4e8f-b886-1d969bd376ea,exposed,2020-07-08,14,Samsung SM-A202F,6,Samsung Internet,0,0,Wednesday


In [298]:
df_sample['day_of_week'].value_counts()

day_of_week
Friday       1636
Thursday      698
Wednesday     692
Monday        284
Tuesday       282
Name: count, dtype: int64

In [299]:
df_sample_gr = df_sample.groupby('test_group').agg({'responded':'sum',
                                                    'id':'nunique'}).reset_index()

df_sample_gr.head()

,test_group,responded,id
0,control,251,1796
1,exposed,288,1796


In [300]:
number_of_successes = df_sample_gr['responded'].tolist()
print('Number of Successes:', number_of_successes)

total_sample_sizes = df_sample_gr['id'].tolist()
print('Total Sample Sizes:',total_sample_sizes)

Number of Successes: [251, 288]
Total Sample Sizes: [1796, 1796]


In [301]:
test_stat, p_value = proportions_ztest(number_of_successes, total_sample_sizes, alternative='smaller')
print('Test Stat:', test_stat)
print('P-value:', p_value)

Test Stat: -1.7286700325308286
P-value: 0.04193408384759775


## Conclusion and Recommendation

### Acceptance Rate (Obsolete - Not Enough Samples)

In [149]:
# df_acceptance = df[df['responded'] == 1].groupby('test_group').agg({'accepted':'sum', 'id':'nunique'}).reset_index()
# df_acceptance
# df_acceptance['proportion'] = (df_acceptance['accepted']) / df_response['id']
# df_acceptance

,test_group,accepted,id,proportion
0,control,264,586,0.064849
1,exposed,308,657,0.076885
